In [31]:
import tensorflow as tf
import pathlib
import data
import time

In [32]:
def getImageDataGenerators(train_data_directory, validation_data_directory):
    train_image_generator      = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
    validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
    
    train_data_gen = train_image_generator.flow_from_directory(directory     = str(train_data_directory),
                                                               target_size   = (299, 299),
                                                               interpolation = "lanczos")
    
    validation_data_gen = validation_image_generator.flow_from_directory(directory    = str(validation_data_directory),
                                                                        target_size   = (299, 299),
                                                                        interpolation = "lanczos")
    return train_data_gen, validation_data_gen

In [33]:
def getModel(numClasses):
    base_model = tf.keras.applications.InceptionV3(input_shape = (299, 299, 3), 
                                                   include_top = False, 
                                                   weights     = 'imagenet')
    base_model.trainable = False
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(numClasses, activation='softmax')
    
    model = tf.keras.Sequential([base_model, 
                                 global_average_layer, 
                                 prediction_layer])
    
    model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.0001),
                  loss      = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics   = ['accuracy'])
    return model

In [34]:
def fineTuneModel(model):
    base_model = model.layers[0]
    base_model.trainable = True
    
    for layer in base_model.layers[:300]:
        layer.trainable = False
        
    model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.00001),
                  loss      = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics   = ['accuracy'])
    return model

In [35]:
def trainModel(model, initial_epoch, epochs, train_data_gen, validation_data_gen, callbacks):
    history = model.fit(train_data_gen,
                        initial_epoch   = initial_epoch, 
                        epochs          = epochs, 
                        validation_data = validation_data_gen,
                        callbacks       = callbacks)
    return model, history

In [39]:
def main():
    d = data.Data()
    numClasses = d.numClasses
    train_data_directory      = pathlib.Path(r"D:\ActionRecognition\Train")
    validation_data_directory = pathlib.Path(r"D:\ActionRecognition\Test")
    train_data_gen, validation_data_gen = getImageDataGenerators(train_data_directory, validation_data_directory)
    
    modelCheckpointDirectory = pathlib.Path(r"D:\ActionRecognition\Callbacks\CNN\ModelCheckpoint")
    tensorboardDirectory = pathlib.Path(r"D:\ActionRecognition\Callbacks\CNN\Tensorboard")
    
    modelCheckpoint = tf.keras.callbacks.ModelCheckpoint(filepath = str(modelCheckpointDirectory/'CNN_{epoch:03d}_{val_loss:.2f}'),
                                                         save_best_only = True)
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir = str(tensorboardDirectory/f'{int(time.time())}'))
    
    model = getModel(numClasses)
    trained_model, history = trainModel(model, 0, 1, train_data_gen, validation_data_gen, [])
    fine_tuned_model = fineTuneModel(trained_model)
    trained_model, history_fine = trainModel(fine_tuned_model, 1, 2, train_data_gen, validation_data_gen, [modelCheckpoint, tensorboard])

In [40]:
main()

Found 287 images belonging to 3 classes.
Found 117 images belonging to 3 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 9 steps, validate for 4 steps
8/9 [=========================>....] - ETA: 8s - loss: 1.1029 - accuracy: 0.3647 WARNING:tensorflow:From C:\Users\saddi\Anaconda3\envs\Action_Recognition\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: D:\ActionRecognition\Callbacks\CNN\ModelCheckpoint\CNN_001_1.12\assets
9/9 [==============================] - 198s 22s/step - loss: 1.0992 - accuracy: 0.3833 - val_loss: 1.1219 - val_accuracy: 0.2906


In [8]:

# import time module and use it in pathname!!!!
modelCheckpointDirectory = pathlib.Path(r"D:\ActionRecognition\Callbacks\CNN\ModelCheckpoint")
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint(filepath = modelCheckpointDirectory/'CNN_{epoch:03d}_{val_loss:.2f}_{int(time.time())}',
                                                     save_best_only = True)

tensorboardDirectory = pathlib.Path(r"D:\ActionRecognition\Callbacks\CNN\Tensorboard")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir = tensorboardDirectory/'{int(time.time())}')

In [30]:
my_str = f"string_{int(time.time())}"

test_dir      = str(pathlib.Path(r"D:\ActionRecognition\Train")/f'{int(time.time())}')
print(test_dir)
print(type(test_dir))

D:\ActionRecognition\Train\1586712560
<class 'str'>
